In [1]:
import os
import time

import requests
from dotenv import load_dotenv

from youtube_summarizer.summarizer import summarize_video

load_dotenv()

# YouTube URL
youtube_url = "https://youtu.be/pmdiKAE_GLs"

# Step 1: Get transcript from Scrape Creators API
print(f"🔗 Fetching transcript from Scrape Creators API...")
start_time = time.time()

api_key = os.getenv("SCRAPECREATORS_API_KEY")
if not api_key:
    raise ValueError("SCRAPECREATORS_API_KEY environment variable is required")

url = f"https://api.scrapecreators.com/v1/youtube/video?url={youtube_url}&get_transcript=true"
headers = {"x-api-key": api_key}

response = requests.get(url, headers=headers)
response.raise_for_status()

result = response.json()
transcript = result.get("transcript_only_text", "")

elapsed = time.time() - start_time
print(f"✅ Transcript fetched in {elapsed:.2f}s ({len(transcript)} characters)")

if not transcript:
    raise ValueError("No transcript found in API response")

# Step 2: Summarize using summarize_video() with transcript
print(f"\n🤖 Starting summarization...")
analysis = summarize_video(transcript)

# Display results
print("\n" + "=" * 80)
print("ANALYSIS RESULTS")
print("=" * 80)
print(f"\n📌 Title: {analysis.title}")
print(f"\n📝 Summary: {analysis.summary[:200]}...")
print(f"\n🎯 Takeaways ({len(analysis.takeaways)}):")
for i, takeaway in enumerate(analysis.takeaways, 1):
    print(f"  {i}. {takeaway}")
print(f"\n📊 Key Facts ({len(analysis.key_facts)}):")
for i, fact in enumerate(analysis.key_facts, 1):
    print(f"  {i}. {fact}")
print(f"\n📚 Chapters ({len(analysis.chapters)}):")
for i, chapter in enumerate(analysis.chapters, 1):
    print(f"  {i}. {chapter.header}")
    print(f"     {chapter.summary[:100]}...")
print(f"\n🔑 Keywords: {', '.join(analysis.keywords)}")

🔗 Fetching transcript from Scrape Creators API...
✅ Transcript fetched in 22.80s (36917 characters)

🤖 Starting summarization...
📝 Using model: x-ai/grok-4-fast
📝 Super node mode: generating initial analysis. Transcript length: 36917
📝 Text preview: Palunteer. Congratulations all Palunteer shareholders. This is not a video I should be recording right now. I was halfway done preparing a video for the main channel called four stocks I'm buying now ...
📊 LangChain analysis completed
🔍 Performing quality check with LangChain...
🔍 Using model: x-ai/grok-4-fast
📈 LangChain quality breakdown:
Completeness: Pass - The analysis covers all key elements from earnings results, segment breakdowns, profitability, valuation, projections, and strategic aspects, indicating comprehensive consideration of the transcript content.
Structure: Refine - The output follows the desired structure with title, summary, takeaways, key facts, chapters, and keywords; however, some chapter summaries are slightly below